# Production ready crosslight detection

### Lets create our model and load the weights

In [ ]:
import display

# Display number
display.five()

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import img_to_array, load_img # Do I really need this?

from picamera import PiCamera, array
from time import sleep
from numpy import around

# Display number
display.four()

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(112, 112, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
binary_crossentropy
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Display number
display.three()

model.load_weights('/home/pi/repositories/pren/models/crosslight.h5')

# Display number
display.two()

### Raspberry camera logic

In [ ]:
with PiCamera() as camera:
    camera.resolution = (112, 112)
    
    # Sleep to let camera adjust to the light
    sleep(1)
    counter = 0
    
    # Display number
    display.one()
    with array.PiRGBArray(camera) as output:
        while counter < 1:
            camera.capture(output, 'rgb')
            img = output.array.astype(float)
            img = img.reshape((1,) + img.shape)
            prediction = around(model.predict(img, 3))[0]

            if prediction[1] > 0.90:
                display.zero()
            if prediction[0] > 0.90:
                counter += 1
                
            output.truncate(0)
        print("go")